# Формирование наборов данных

## Предметный корпус

###  Парсер предметного корпуса предложений

> Использован для извлечения англо-русских фраз из канала [Английский для программиста](https://t.me/english_easily) в Telegram

In [16]:
from bs4 import BeautifulSoup
import pandas as pd

In [44]:
eng_sym = set([chr(i) for i in range(65, 91)] + [chr(i) for i in range(97, 123)])
rus_sym = set([chr(i) for i in range(1040, 1104)])

In [173]:
def get_soup(file):
    '''Получаем чистый суп'''
    with open(file, 'r') as f:
        contents = f.read()    
        contents = contents.replace('<br><br>', '<br>')
        contents = contents.replace('—', '-') 
        contents = contents.replace('—', '-') 
        contents = contents.replace('–', '-')     
        contents = contents.replace('<br>- ', '-')
        contents = contents.replace('.<br>- ', '.-')    
        contents = contents.replace('. <br> - ', '.-')    
        contents = contents.replace('-<br>', '-')    
        contents = contents.replace(' - <br>', '-')    
        contents = contents.replace('.<br> - ', '-')        
        contents = contents.replace('.<br></strong>- ', '-')        
        contents = contents.replace('<br></strong>- ', '-')
        contents = contents.replace('.<br>-', '-')
    return BeautifulSoup(contents, 'lxml')    


def messages_list(soup):
    '''Список всех сообщений'''
    return soup.find_all('div', {
        'class': 'message default clearfix'
        })


def key_words(text):
    '''Список ключевых слов'''
    result = BeautifulSoup(text, 'lxml').find_all('strong')
    result = set(_.get_text().lower().strip() for _ in result)
    if len(result) == 2:        
        return tuple(result)
    eng, rus = [], []
    for word in sorted(result):
        if len(set(word) & eng_sym) > 1:
            eng.append(word)
        else:                
            rus.append(word)
    eng = ", ".join(eng) if eng else None
    rus = ", ".join(rus) if rus else None
    return eng, rus


def vocab(text):
    '''Список фраз'''
    result = None 
    data = BeautifulSoup(text, 'lxml').get_text()
    
    while data[0].isdigit():
        data = data[1:]
    try:
        while data[0] == '.' or data[0] == ')':
            data = data[1:]
    except IndexError:
        return
    if data.count('-') == 1:
        result = list(map(str.strip, data.split('-')))
        return result if set(result[0]) & eng_sym else None
    elif data.count('-') > 1:
        tmp = data.split('-')        
        eng, rus = [], []
        for sent in tmp:
            if len(set(sent) & eng_sym) > int(len(set(sent)) * 0.5) and not set(sent) & rus_sym:                
                eng.append(sent)
            else:                
                rus.append(sent)
        result = ['-'.join(eng).strip(), '-'.join(rus).strip()]
    elif data.count('-') < 1:
        return # Остались, те, которые не содержат тире между парой предложений либо рекламу
    return result
        

def frame(soup):
    '''Датафрейм с данными'''
    result = []
    for message in messages_list(soup):
        message = message.find('div', {'class':"text"})
        data = [_ for _ in str(message).split('<br/>') if _ and '<strong>' in _ and '</strong>' in _]
        data[0] = data[0].split('</a>')[-1]
        data[-1] = data[-1].split('</div>')[0].strip()
        data[0] = data[0].replace('??', '')
        data[0] = data[0].replace('</em>', '')
        for text in data:
            texts = []
            if text.startswith('</strong>'):
                text = text[len('</strong>'):]
            if text.startswith('<strong>') and text[len('<strong>')].isdigit():
                text = text[len('<strong>'):]
                while text[0].isdigit():
                    text = text[1:]
                text = text[len('</strong>'):]
            if vocab(text) and vocab(text)[1]:
                texts.extend(vocab(text))
                texts.extend(key_words(text))
                result.append(texts)
    return pd.DataFrame(result, columns=['en_sent', 'ru_sent', 'en_keys', 'ru_keys'] )


file = 'messages.html'
df = frame(get_soup(file)); df

,en_sent,ru_sent,en_keys,ru_keys
0,Go wash your hands,Иди мой руки.,мой,wash
1,He washes the car,Он моет машину.,моет,washes
2,He washed the car,Он помыл машину.,помыл,washed
3,Wash your hands well,Мой свои руки хорошенько.,мой,wash
4,I hope to see you again,Я надеюсь увидеться с вами снова.,again,снова
...,...,...,...,...
1798,Disable anti-virus firewall and open up port 443.,Отключите брандмауэр для защиты от вирусов и о...,firewall,брандмауэр
1799,We need to find a way to break through that si...,Нам нужно найти способ обойти брандмауэр сайта.,firewall,брандмауэр
1800,Several representatives supported incremental ...,Ряд представителей высказались за поэтапное ра...,incremental development,поэтапное развитие
1801,Incremental development increases affordabilit...,Поэтапное развитие повышает доступность за сче...,incremental development,поэтапное развитие


In [174]:
df.to_excel('prog_vocab.xlsx')

### Выполнена ручная корректировка корпуса
> Сохранен файл ```prog_vocab.xlsx```

In [180]:
df = pd.read_excel('corrected_vocab.xlsx'); df.head()

,en_sent,ru_sent,en_keys,ru_keys
0,Please contact us as soon as possible,"пожалуйста, свяжитесь с нами как можно скорее.",contact,свяжитесь
1,If you would like to have an additional inform...,если вы хотите получить дополнительную информа...,contact,свяжитесь
2,This task is difficult for me.,Это задание трудное для меня.,задание,task
3,Our task is to increase channel activity in th...,"Наша задача состоит в том, чтобы увеличить акт...",задача,task
4,My employee never completes tasks on time.,Мой сотрудник никогда не делает задания в срок.,задания,tasks


In [181]:
df.shape

(1038, 4)

In [182]:
[i for i in df['en_sent']][:5]

['Please contact us as soon as possible',
 'If you would like to have an additional information, please contact me',
 'This task is difficult for me.',
 'Our task is to increase channel activity in the coming month.',
 'My employee never completes tasks on time.']

## Пакетные переводы

### Перевод предметного корпуса

In [ ]:
df = pd.read_excel('corrected_vocab.xlsx')

In [183]:
import requests
from sentence_splitter import SentenceSplitter, split_text_into_sentences
url = 'http://localhost:5000/api/translation'
headers = {'accept': 'application/json',
           'Content-Type': 'application/json',
}
request = lambda x: '{ \"text\": \"'+ str(x) +'\", \"sourceLanguage\": \"en\", \"targetLanguage\": \"ru\"}'
response = lambda z: requests.post(url, headers=headers, data=request(z)).json()['message']

In [184]:
%time result = [response(i.encode('latin-1', 'ignore').decode("utf-8")) for i in df['en_sent']]

Wall time: 39min 24s


### Добавление перевода в предметный копус

In [185]:
len(df['en_sent']), len(df['ru_sent']), len(result)

(1038, 1038, 1038)

In [186]:
short_df = df[['en_sent', 'ru_sent']]
short_df

,en_sent,ru_sent
0,Please contact us as soon as possible,"пожалуйста, свяжитесь с нами как можно скорее."
1,If you would like to have an additional inform...,если вы хотите получить дополнительную информа...
2,This task is difficult for me.,Это задание трудное для меня.
3,Our task is to increase channel activity in th...,"Наша задача состоит в том, чтобы увеличить акт..."
4,My employee never completes tasks on time.,Мой сотрудник никогда не делает задания в срок.
...,...,...
1033,Disable anti-virus firewall and open up port 443.,Отключите брандмауэр для защиты от вирусов и о...
1034,We need to find a way to break through that si...,Нам нужно найти способ обойти брандмауэр сайта.
1035,Several representatives supported incremental ...,Ряд представителей высказались за поэтапное ра...
1036,Incremental development increases affordabilit...,Поэтапное развитие повышает доступность за сче...


In [187]:
short_df.loc[:, 'api'] = result

C:\Users\e.boltachev\Anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [188]:
short_df

,en_sent,ru_sent,api
0,Please contact us as soon as possible,"пожалуйста, свяжитесь с нами как можно скорее.","Пожалуйста, свяжитесь с нами как можно скорее."
1,If you would like to have an additional inform...,если вы хотите получить дополнительную информа...,Если вы хотите получить дополнительную информа...
2,This task is difficult for me.,Это задание трудное для меня.,Для меня это трудная задача.
3,Our task is to increase channel activity in th...,"Наша задача состоит в том, чтобы увеличить акт...","Наша задача состоит в том, чтобы увеличить акт..."
4,My employee never completes tasks on time.,Мой сотрудник никогда не делает задания в срок.,Мой сотрудник никогда не выполняет задания вов...
...,...,...,...
1033,Disable anti-virus firewall and open up port 443.,Отключите брандмауэр для защиты от вирусов и о...,Отключить антивирусный файервол и открыть порт...
1034,We need to find a way to break through that si...,Нам нужно найти способ обойти брандмауэр сайта.,Нам нужно найти способ прорваться через файерв...
1035,Several representatives supported incremental ...,Ряд представителей высказались за поэтапное ра...,Ряд представителей высказались в поддержку пос...
1036,Incremental development increases affordabilit...,Поэтапное развитие повышает доступность за сче...,Повышенное развитие повышает доступность за сч...


In [189]:
short_df.head()

,en_sent,ru_sent,api
0,Please contact us as soon as possible,"пожалуйста, свяжитесь с нами как можно скорее.","Пожалуйста, свяжитесь с нами как можно скорее."
1,If you would like to have an additional inform...,если вы хотите получить дополнительную информа...,Если вы хотите получить дополнительную информа...
2,This task is difficult for me.,Это задание трудное для меня.,Для меня это трудная задача.
3,Our task is to increase channel activity in th...,"Наша задача состоит в том, чтобы увеличить акт...","Наша задача состоит в том, чтобы увеличить акт..."
4,My employee never completes tasks on time.,Мой сотрудник никогда не делает задания в срок.,Мой сотрудник никогда не выполняет задания вов...


#### Сохраняем результат перевода в файл ```translated_vocab.xlsx```

In [ ]:
short_df.to_excel('translated_vocab.xlsx')

#### Создаем файл ```diff_translated_vocab.xlsx```, в котором переводы отличаются от эталона

In [191]:
diff = short_df[short_df['ru_sent'] != short_df['api']]

In [192]:
diff.to_excel('diff_translated_vocab.xlsx')

In [193]:
diff

,en_sent,ru_sent,api
0,Please contact us as soon as possible,"пожалуйста, свяжитесь с нами как можно скорее.","Пожалуйста, свяжитесь с нами как можно скорее."
1,If you would like to have an additional inform...,если вы хотите получить дополнительную информа...,Если вы хотите получить дополнительную информа...
2,This task is difficult for me.,Это задание трудное для меня.,Для меня это трудная задача.
3,Our task is to increase channel activity in th...,"Наша задача состоит в том, чтобы увеличить акт...","Наша задача состоит в том, чтобы увеличить акт..."
4,My employee never completes tasks on time.,Мой сотрудник никогда не делает задания в срок.,Мой сотрудник никогда не выполняет задания вов...
...,...,...,...
1033,Disable anti-virus firewall and open up port 443.,Отключите брандмауэр для защиты от вирусов и о...,Отключить антивирусный файервол и открыть порт...
1034,We need to find a way to break through that si...,Нам нужно найти способ обойти брандмауэр сайта.,Нам нужно найти способ прорваться через файерв...
1035,Several representatives supported incremental ...,Ряд представителей высказались за поэтапное ра...,Ряд представителей высказались в поддержку пос...
1036,Incremental development increases affordabilit...,Поэтапное развитие повышает доступность за сче...,Повышенное развитие повышает доступность за сч...


## Работа с обыденными фразами

In [225]:
df = pd.read_excel('normal.xlsx')

In [226]:
df.head()

,en_sent,ru_sent
0,Good morning!,Доброе утро!
1,Did the alarm clock go off?,Будильник звонил?
2,It's time to get up.,Время вставать!
3,Get up soon.,Вставай быстрее.
4,Are you awake?,Ты проснулся?


In [196]:
import requests
from sentence_splitter import SentenceSplitter, split_text_into_sentences
url = 'http://localhost:5000/api/translation'
headers = {'accept': 'application/json',
           'Content-Type': 'application/json',
}
request = lambda x: '{ \"text\": \"'+ str(x) +'\", \"sourceLanguage\": \"en\", \"targetLanguage\": \"ru\"}'
response = lambda z: requests.post(url, headers=headers, data=request(z)).json()['message']

In [197]:
%time result = [response(i.replace('"', '\\"').encode('ascii', errors='ignore').decode("utf-8")) for i in df['en_sent']]

Wall time: 1h 37min 32s


In [199]:
result[:10]

['Доброе утро!',
 'Сработал будильник?',
 'Пора вставать.',
 'Скоро вставайте.',
 'Ты не спишь?',
 'Тебе плохо?',
 'Вы хорошо спали?',
 'Выключи будильник.',
 'Наконец-то ты встал!',
 'Хороший день!']

In [200]:
df.loc[:, 'api'] = result

In [230]:
df.head()

,en_sent,ru_sent,api
0,Good morning!,Доброе утро!,Доброе утро!
1,Did the alarm clock go off?,Будильник звонил?,Сработал будильник?
2,It's time to get up.,Время вставать!,Пора вставать.
3,Get up soon.,Вставай быстрее.,Скоро вставайте.
4,Are you awake?,Ты проснулся?,Ты не спишь?


In [231]:
df.shape

(2664, 3)

### Сохраняем результат перевода в файл ```translated_normal.xlsx```

In [215]:
df.to_excel('translated_normal.xlsx')

### Создаем файл ```diff_translated_normal.xlsx```, в котором переводы отличаются от эталона

In [232]:
diff = df[df['ru_sent'] != df['api']]

In [233]:
diff.head()

,en_sent,ru_sent,api
1,Did the alarm clock go off?,Будильник звонил?,Сработал будильник?
2,It's time to get up.,Время вставать!,Пора вставать.
3,Get up soon.,Вставай быстрее.,Скоро вставайте.
4,Are you awake?,Ты проснулся?,Ты не спишь?
5,Are you feeling sick?,Ты болен?,Тебе плохо?


In [234]:
diff.shape

(2096, 3)

In [235]:
diff.to_excel('diff_translated_normal.xlsx')

## Тестовый корпус для используемой модели

###  Парсер тестового корпуса предложений

In [21]:
import codecs

with codecs.open("test_en-ru_opus_1-src_2-tgt_3-translation.txt", "r", "utf_8_sig") as f:
    text = f.readlines()
text = [(src.strip(), tgt.strip()) for src, tgt in zip(text[0::4], text[1::4])]
text = pd.DataFrame(text, columns=['en', 'ru'])
text.head()

,en,ru
0,Have you ever been to Switzerland?,Ты уже бывал в Швейцарии?
1,She learned quickly.,Она быстро училась.
2,No one tried to stop me.,Никто не пытался меня остановить.
3,Guilds were an important part of society in th...,Гильдии были важной частью общества в Средние ...
4,You say that it is your custom to burn widows....,"Вы говорите, что сжигать вдов - ваш обычай. Оч..."


In [22]:
text.to_excel('test_opus_en-ru_dataset.xlsx')